Sample input: Agia Triada, Καραισκάκη, Οπλαρχηγού Σεχιώτη, Καλαβρύτων

In [2]:
import json
import folium
from geopy.distance import geodesic

# Read the JSON data from a file
with open('../DataSets/fixedJson.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Function to get user input for road names (comma-separated)
def get_road_names():
    names_input = input("Enter the names of roads (comma-separated, in Greek if applicable): ")
    road_names = [name.strip() for name in names_input.split(",")]  # Split by comma and remove extra spaces
    return road_names

# Get the road names from the user
target_road_names = get_road_names()

# Function to find the nearest point to connect roads
def find_nearest_point(start, candidates):
    nearest_point = None
    min_distance = float('inf')
    
    for point in candidates:
        distance = geodesic(start, (point[1], point[0])).meters  # Geodesic distance between points
        if distance < min_distance:
            min_distance = distance
            nearest_point = point
            
    return nearest_point

# Define a function to find and create a route
def create_route(data, target_road_names):
    named_roads = []
    
    # Check for roads with the specified names
    for feature in data.get("features", []):  # Use get to avoid KeyError
        if "properties" in feature and "geometry" in feature:
            road_name = feature["properties"].get("name")
            if road_name in target_road_names:  # Match the specified road names
                named_roads.append(feature)

    # Ensure at least 4 roads are found
    if len(named_roads) < 4:
        print("Not enough named roads found.")
        return None
    
    # Create a route by connecting the roads in sequence
    route_coordinates = []
    current_road = named_roads[0]["geometry"]["coordinates"]
    route_coordinates.extend(current_road)  # Start with the first road

    # Iterate through the remaining roads and connect them based on nearest points
    for next_road in named_roads[1:]:
        next_coords = next_road["geometry"]["coordinates"]
        # Find the nearest point in the current road to connect to the next road
        last_point = route_coordinates[-1]
        nearest_point = find_nearest_point(last_point, next_coords)
        
        # Adjust the order of the next road coordinates so they connect properly
        if nearest_point != next_coords[0]:
            next_coords.reverse()
        
        route_coordinates.extend(next_coords)  # Add the next road's coordinates to the route
    
    # Return the route as a GeoJSON FeatureCollection structure
    route = {
        "type": "FeatureCollection",
        "features": [
            {
                "type": "Feature",
                "geometry": {
                    "type": "LineString",
                    "coordinates": route_coordinates
                },
                "properties": {
                    "name": "Route connecting specified roads",
                    "description": "Route connecting roads: " + ", ".join(target_road_names)
                }
            }
        ]
    }
    
    return route

# Create the route
route = create_route(data, target_road_names)

# Save the route to a new JSON file in ArcGIS-compatible format
if route:
    with open('dummy_trips.json', 'w', encoding='utf-8') as f:
        json.dump(route, f, ensure_ascii=False, indent=4)
    print("Route saved to dummy_trips.json in ArcGIS-compatible format.")

# Create a Folium map centered around Tripoli, Greece
map_center = [37.5006173, 22.3953491]  # Center the map around Tripoli
m = folium.Map(location=map_center, zoom_start=13)

# Add the routes found in the data
for feature in data.get("features", []):  # Use get to avoid KeyError
    if "geometry" in feature:  # Ensure geometry exists
        coords = feature["geometry"]["coordinates"]
        folium.PolyLine(locations=[(coord[1], coord[0]) for coord in coords], color="blue", weight=2, opacity=0.5).add_to(m)

# Add markers for each road feature that matches the criteria
for feature in data.get("features", []):  # Use get to avoid KeyError
    if "properties" in feature and "geometry" in feature:  # Ensure both keys exist
        road_name = feature["properties"].get("name", "Unnamed Road")  # Default if name is missing
        if road_name in target_road_names:  # Only add markers for the specified roads
            coords = feature["geometry"]["coordinates"][0]  # Use the first coordinate for the marker
            folium.Marker(location=(coords[1], coords[0]), popup=road_name).add_to(m)

# Save map to HTML file
m.save('tripoli_roads_map.html')

# To display the map in Jupyter Notebook (uncomment below if using a Jupyter environment)
m


Route saved to dummy_trips.json in ArcGIS-compatible format.


In [1]:
import json
import math

# Helper function to calculate distance between two coordinates using the Haversine formula
def haversine(coord1, coord2):
    R = 6371000  # Radius of the Earth in meters
    lat1, lon1 = math.radians(coord1[1]), math.radians(coord1[0])
    lat2, lon2 = math.radians(coord2[1]), math.radians(coord2[0])
    
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    return R * c

# Function to check if two roads are "connected" based on proximity of their end/start points
def are_connected(road1, road2, default_threshold=50):  # Default threshold
    # Determine the initial threshold
    threshold = default_threshold
    
    # Get road names for comparison
    road1_name = get_road_name(road1['properties'])
    road2_name = get_road_name(road2['properties'])
    
    # Increase threshold if road names match
    if road1_name and road2_name and road1_name == road2_name:
        threshold += 50  # Increase threshold by 50 meters if names match
        print(f"Threshold increased to {threshold} meters for roads: '{road1_name}' and '{road2_name}'")

    # Coordinates of the roads
    road1_coords = road1['geometry']['coordinates']
    road2_coords = road2['geometry']['coordinates']
    
    # Check if the last point of road1 is close to the first point of road2
    connected_last_first = haversine(road1_coords[-1], road2_coords[0]) < threshold
    # Check if the first point of road1 is close to the last point of road2
    connected_first_last = haversine(road1_coords[0], road2_coords[-1]) < threshold

    print(f"Checking connection between '{road1_name}' and '{road2_name}': {connected_last_first or connected_first_last}")
    
    return connected_last_first or connected_first_last

# Function to extract road names, giving priority to 'name' and ignoring duplicates
def get_road_name(properties):
    if not properties:
        return ""  # Return empty string if properties are missing
    
    if 'name' in properties:
        return properties['name']  # Prefer 'name' field for display
    elif 'int_name' in properties:
        return properties['int_name']
    elif 'ref' in properties:
        return properties['ref']
    elif 'old_ref' in properties:
        return properties['old_ref']
    return ""  # Return empty string if no relevant property is found

# Function to find connected roads based on input names and coordinates
def find_connected_roads(road_names, road_data):
    connected_roads = []
    
    # Filter roads with valid properties and any matching names
    roads_to_route = [
        road for road in road_data['features']
        if 'properties' in road and any(name in get_road_name(road['properties']) for name in road_names if get_road_name(road['properties']))
    ]
    
    if not roads_to_route:
        print("No matching roads found")
        return connected_roads

    # Start with the first road
    connected_roads.append(roads_to_route[0])
    roads_to_route.pop(0)

    # Attempt to connect remaining roads
    while roads_to_route:
        current_road = connected_roads[-1]
        found_connection = False  # Flag to track if any connection was found

        for i, road in enumerate(roads_to_route):
            # Check if roads are connected based on proximity (considering both ends of the roads)
            if are_connected(current_road, road):
                print(f"Connected: {get_road_name(current_road['properties'])} <--> {get_road_name(road['properties'])}")
                connected_roads.append(road)
                roads_to_route.pop(i)  # Remove connected road from the list
                found_connection = True  # Set the flag to true
                break  # Exit the inner loop to restart checking with the new current road

        if not found_connection:
            # If no connection was found in this iteration, break the loop
            print(f"No more connections found for '{get_road_name(current_road['properties'])}'.")
            break
    
    return connected_roads

# Load the entire JSON file and handle missing properties gracefully
def load_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        road_data = json.load(file)
    return road_data

# Function to ensure unique road names in the output and ignore duplicates
def get_unique_road_names(connected_roads):
    seen_names = set()
    unique_road_names = []
    
    for road in connected_roads:
        road_name = get_road_name(road['properties'])
        if road_name and road_name not in seen_names:
            seen_names.add(road_name)
            unique_road_names.append(road_name)
    
    return unique_road_names

# Main function to run the process
def main():
    # Load the JSON file (assuming it's saved locally)
    road_data = load_json('../DataSets/fixedJson.json')  # Replace 'roads_data.json' with your actual file path
    
    # Input: Comma-separated road names from user
    input_road_names = "Καλαβρύτων, ΕΠ3"
    road_names = [name.strip() for name in input_road_names.split(",")]

    # Find the connected roads
    connected_roads = find_connected_roads(road_names, road_data)

    # Output unique connected road names
    unique_road_names = get_unique_road_names(connected_roads)
    if unique_road_names:
        print(f"Connected roads: {', '.join(unique_road_names)}")
    else:
        print("No roads could be connected.")

# Run the main function
if __name__ == "__main__":
    main()


Checking connection between 'Καλαβρύτων' and 'ΕΠ3': False
Threshold increased to 100 meters for roads: 'Καλαβρύτων' and 'Καλαβρύτων'
Checking connection between 'Καλαβρύτων' and 'Καλαβρύτων': False
Threshold increased to 100 meters for roads: 'Καλαβρύτων' and 'Καλαβρύτων'
Checking connection between 'Καλαβρύτων' and 'Καλαβρύτων': False
Threshold increased to 100 meters for roads: 'Καλαβρύτων' and 'Καλαβρύτων'
Checking connection between 'Καλαβρύτων' and 'Καλαβρύτων': False
Threshold increased to 100 meters for roads: 'Καλαβρύτων' and 'Καλαβρύτων'
Checking connection between 'Καλαβρύτων' and 'Καλαβρύτων': False
Threshold increased to 100 meters for roads: 'Καλαβρύτων' and 'Καλαβρύτων'
Checking connection between 'Καλαβρύτων' and 'Καλαβρύτων': False
Threshold increased to 100 meters for roads: 'Καλαβρύτων' and 'Καλαβρύτων'
Checking connection between 'Καλαβρύτων' and 'Καλαβρύτων': False
Threshold increased to 100 meters for roads: 'Καλαβρύτων' and 'Καλαβρύτων'
Checking connection between